<a href="https://colab.research.google.com/github/Harsh-Prajapati54/PyTorch-Learning-Journey/blob/master/06_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Recurrent Neural Network  